In [1]:
#!pip install --upgrade google-api-python-client
#!pip install oauth2client

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import apikey

In [2]:
dev_key = apikey.api_key
api_service_name = "youtube"
api_ver = "v3"

In [3]:
youtube = build(api_service_name, api_ver, developerKey = dev_key)

In [4]:
search_response = youtube.search().list(
    q = "공부하는섭이",
    order = "relevance",
    part = "snippet",
    maxResults = 50
    ).execute()

In [5]:
channel_id = search_response['items'][0]['id']['channelId']

In [6]:
playlists = youtube.playlists().list(
    channelId = channel_id,
    part = "snippet",
    maxResults = 20
    ).execute()

In [7]:
import pandas as pd

ids=[]
titles=[]
for i in playlists['items']:
    ids.append(i['id'])
    titles.append(i['snippet']['title'])
    
df=pd.DataFrame([ids,titles]).T
df.columns=['PlayLists','Titles']

df.head(5)

,PlayLists,Titles
0,PLfkfgvn_kFAU6nfQ-zL_JV8D_SWeHVuRV,Shorts
1,PLfkfgvn_kFAX8jvsuRcoFp6CFPAUehok8,일상 Vlog
2,PLfkfgvn_kFAV4YksRjqGU_H9dWmMRylSG,Q&A / 정보 제공


In [14]:
#영상 list
dtcu=df['PlayLists'][1]
playlist_videos=youtube.playlistItems().list(
    playlistId=dtcu,
    part='snippet',
    maxResults=50)

playlistitems_list_response=playlist_videos.execute()

video_names=[]
video_ids=[]
date=[]

for v in playlistitems_list_response['items']:
    video_names.append(v['snippet']['title'])
    video_ids.append(v['snippet']['resourceId']['videoId'])
    date.append(v['snippet']['publishedAt'])
    
vdf=pd.DataFrame([date,video_names,video_ids]).T
vdf.columns=['Date','Title','IDS']
vdf



,Date,Title,IDS
0,2022-08-20T20:26:52Z,미국 대학원생 여름학기 일상 Vlog - 벌써 1년?,fLA0abcqyTE
1,2022-08-20T20:26:52Z,미국 인턴 구르는 Vlog - 본격 예비 외노자(되고싶다)의 미국 회사 체험,aBtYmx-AhlU
2,2022-08-20T20:26:52Z,여름방학 맞아 푹 삶아진(?) 미국 대학원생 Vlog - 응 몰라 일단 쉴거야,frxWAxQvPcw
3,2022-08-20T20:26:52Z,기말고사까지 우당탕탕 치른 미국 대학원생 Vlog - 이겨내는 것이 아니라 버티는 것이다,5TQPtOVdr9Y
4,2022-08-20T20:26:52Z,또 첫 학기 어찌저찌 보낸 미국 대학원생 Vlog - 달라지기 위한 나날들,am2TFveOX08
5,2022-08-20T20:26:52Z,미국 대학원생 방학에 좀 숨 돌리는 Vlog - 아니 왜 벌써 개강인데.....?,mtPM6m3J0QU
6,2022-08-20T20:26:52Z,미국 대학원생 연말 홀로 즐기는 Vlog - 혼자서도 연말 잘 보내는 방법,fRLiHCW-HXA
7,2022-08-20T20:26:52Z,해커톤 해도 됩니까? - 미국 대학원생 첫 해커톤 Vlog,sAVPn9qOdcc
8,2022-08-20T20:26:52Z,미국 대학원생 첫학기 드디어 끝난 Vlog - 해방이니?,7DaoE5EUDIw
9,2022-08-20T20:26:52Z,미국 대학원생 중간고사 끝난 Vlog - 도랏나 왜 똑같은데,NpXESg1GqRk


In [20]:
import re

category_id=[]
views=[]
likes=[]
comments=[]
mins=[]
seconds=[]
title=[]
date=[]

for u in range(len(vdf)):
    request=youtube.videos().list(
    part='snippet,contentDetails,statistics',
    id=vdf['IDS'][u])
    
    response=request.execute()
    
    if response['items']==[]:
        ids.append('-')
        category_id.append('-')
        views.append('-')
        likes.append('-')
        comments.append('-')
        date.append('-')
        
    else :
        title.append(response['items'][0]['snippet']['title'])
        category_id.append(response['items'][0]['snippet']['categoryId'])
        views.append(response['items'][0]['statistics']['viewCount'])
        likes.append(response['items'][0]['statistics']['likeCount'])
        comments.append(response['items'][0]['statistics']['commentCount'])
        date.append(response['items'][0]['snippet']['publishedAt'])
        
dtcu_df=pd.DataFrame([title,category_id,views,likes,comments,date]).T
dtcu_df.columns=['title','category_id','views','likes','comments','date']
dtcu_df

,title,category_id,views,likes,comments,date
0,미국 대학원생 여름학기 일상 Vlog - 벌써 1년?,22,465,51,26,2022-07-31T11:00:31Z
1,미국 인턴 구르는 Vlog - 본격 예비 외노자(되고싶다)의 미국 회사 체험,22,1444,46,38,2022-06-18T23:58:09Z
2,여름방학 맞아 푹 삶아진(?) 미국 대학원생 Vlog - 응 몰라 일단 쉴거야,22,714,34,18,2022-05-27T23:53:33Z
3,기말고사까지 우당탕탕 치른 미국 대학원생 Vlog - 이겨내는 것이 아니라 버티는 것이다,22,584,50,18,2022-05-12T07:05:45Z
4,또 첫 학기 어찌저찌 보낸 미국 대학원생 Vlog - 달라지기 위한 나날들,27,984,59,44,2022-03-18T07:59:51Z
5,미국 대학원생 방학에 좀 숨 돌리는 Vlog - 아니 왜 벌써 개강인데.....?,27,555,26,20,2022-01-20T08:34:16Z
6,미국 대학원생 연말 홀로 즐기는 Vlog - 혼자서도 연말 잘 보내는 방법,27,605,47,27,2022-01-01T09:33:14Z
7,해커톤 해도 됩니까? - 미국 대학원생 첫 해커톤 Vlog,27,730,34,23,2021-12-25T04:58:33Z
8,미국 대학원생 첫학기 드디어 끝난 Vlog - 해방이니?,27,567,40,25,2021-12-16T10:00:07Z
9,미국 대학원생 중간고사 끝난 Vlog - 도랏나 왜 똑같은데,27,815,36,21,2021-11-23T09:27:13Z
